<p style="text-align:center;">
    <img src="https://raw.githubusercontent.com/skypilot-org/skypilot/master/docs/source/images/skypilot-wide-light-1k.png" width=500>
</p>

# Welcome to SkyPilot! 👋

SkyPilot is a framework for run, manage, and scale AI workloads on any AI infrastructure. SkyPilot offers maximum cost savings, highest GPU availability, managed execution, and multi-region serving.

Manage and use any of your kubernetes clusters or the clouds **easily** and **cost effectively**, without needing any infra expertise.

SkyPilot **is easy to use for AI teams**:
- Quickly spin up compute on your own infra
- Environment and job as code — simple and portable
- Easy job management: queue, run, and auto-recover many jobs

SkyPilot **makes Kubernetes easy for AI & Infra teams**:

- Slurm-like ease of use, cloud-native robustness
- Local dev experience on K8s: SSH into pods, sync code, or connect IDE
- Turbocharge your clusters: gang scheduling, multi-cluster, and scaling

SkyPilot **unifies multiple clusters, clouds, and hardware**:
- One interface to use reserved GPUs, Kubernetes clusters, or 16+ clouds
- [Flexible provisioning](https://docs.skypilot.co/en/latest/examples/auto-failover.html) of GPUs, TPUs, CPUs, with auto-retry
- [Team deployment](https://docs.skypilot.co/en/latest/reference/api-server/api-server.html) and resource sharing

SkyPilot **cuts your cloud costs & maximizes GPU availability**:
* Autostop: automatic cleanup of idle resources
* [Spot instance support](https://docs.skypilot.co/en/latest/examples/managed-jobs.html#running-on-spot-instances): 3-6x cost savings, with preemption auto-recovery
* Intelligent scheduling: automatically run on the cheapest & most available infra

# Learning outcomes 🎯

After completing this notebook, you will be able to:

1. Understand the basic SkyPilot YAML interface (`setup`, `run`).
2. Run a hello world task on a cloud of your choice.
3. Using cursor to connect to your cluster for debugging and development.
4. Terminate the cluster and understand the cluster lifecycle.
5. Run your task seamlessly across different infrastructures.

# How to use this Tutorial

These notebooks serve as an **interactive** introduction to SkyPilot.

There are points in these notebooks where you may need to edit files outside the notebook and open a terminal to run some commands. These points will be highlighted with **two icons**:

### <span style="color:green">[DIY]</span> 📝 - Edit an external file

> **💡 Hint** - Remember to save your file after making any changes!

### <span style="color:green">[DIY]</span> 💻 - Run commands in an interactive terminal window

Use these icons as a hint to know when to switch away from the current notebook and edit a file or open a terminal.

> **💡 Hint** - If you're using jupyter lab, you can create a terminal in your browser by going to `File -> New -> Terminal`

# Preflight checks - connect to SkyPilot API Server

Before we start this tutorial, please use `sky api login` to connect to SkyPilot API Server. A SkyPilot API Server is the backend service that receives and manages all SkyPilot commands—launching clusters, running jobs, and exposing a unified dashboard. In this session, everyone will connect to the same shared API server, so all participants will issue requests through a common control plane. It also serves as a single pane of glass for large-scale deployments, giving you a unified view and management layer across all infrastructure and workloads.

In [ ]:
# Run this cell to connect to a shared SkyPilot API Server
! sky api login -e https://skycamp25.tianxia.me

You should see the following output:

```console
Logged into SkyPilot API server at: https://skycamp25.tianxia.me
└── Dashboard: https://skycamp25.tianxia.me/dashboard
```

To check the status of your SkyPilot API Server, let's run `sky api info` to make sure you connected to the shared API server and it is healthy.

In [ ]:
# Run this cell to check the connection to SkyPilot API Server
! sky api info

You should see something like this:

```console
SkyPilot client version: 1.0.0-dev0, commit: e5b71de1f1a0a21df4077d99df0e4d938d165f14-dirty
Using SkyPilot API server and dashboard: https://skycamp25.tianxia.me
├── Status: ApiServerStatus.HEALTHY, commit: 4e2cfdcdda0b9727a9a80c234f774560e334493b-dirty, version: 0.10.5
├── User: tianxia (dd159187)
└── Endpoint set via /Users/tianxia/.sky/config.yaml
```

Then, let's go to the SkyPilot dashboard to see the status of the API server. Open the link in your browser, and in the `Infra` page, you should have Kubernetes, AWS and GCP marked as `enabled`.

<p style="text-align:center;">
    <img src="../assets/example-infra.png" width=500>
</p>

> **💡 Hint** - SkyPilot also supports Azure, Lambda, RunPod and ~20s of public cloud! Though it is not used in this tutorial, please check out our [docs](https://skypilot.readthedocs.io/en/latest/getting-started/installation.html#cloud-account-setup) on how to setup other cloud account.

# Writing your first SkyPilot Task

A **task** in SkyPilot specifies the command that must be run on the cloud, along with the resources required (e.g. Nvidia/AMD GPUs, TPUs, number of nodes) and any dependencies (e.g., files, packages and libraries).

Tasks in SkyPilot are defined as YAML files. Here is an example:

-----------------------------------
```yaml
# example.yaml
name: example

setup: |
  echo "Run any setup commands here"
  pip install cowsay

run: |
  echo "Hello Stranger!"
  cowsay -t "Moo! SkyPilot!"
```
----------------------------------- 

This defines a task with the following components:

* **setup**: commands that must be run before the task is executed. Here we install any dependencies for the task.

* **run**: commands that run the actual task.

# Launching your first SkyPilot Task with `sky launch`

Once your task YAML is ready, you can run it on the cloud with `sky launch`.

## <span style="color:green">[DIY]</span> 💻 Launch your Sky Task!

**In a terminal window, run:**

-------------------------
```console
sky launch example.yaml -c hello-sky-$SKYPILOT_USER
```
-------------------------

This will take about a minute to run.

> **💡 Hint** - If you're using jupyter lab, you can create a terminal in your browser by going to `File -> New -> Terminal`

You'll notice that SkyPilot will perform multiple actions for you:
#### **1. Find the lowest priced VM instance type across different clouds**

SkyPilot will run its optimizer and present you with the cheapest VM type that fits your resource demand. Kubernetes clusters represent your on-premise infrastructure and considered as zero cost.

```console
root@c4784b202994:/skycamp-tutorial/01_hello_sky# sky launch example.yaml -c hello-sky-$SKYPILOT_USER
YAML to run: example.yaml
Considered resources (1 node):
-------------------------------------------------------------------------------------------------------
 INFRA                                    INSTANCE        vCPUs   Mem(GB)   GPUS   COST ($)   CHOSEN   
-------------------------------------------------------------------------------------------------------
 Kubernetes (amd-devclou...8s-skycamp1)   -               2       2         -      0.00          ✔     
 Kubernetes (amd-devclou...8s-skycamp2)   -               2       2         -      0.00                
 Kubernetes (amd-devclou...8s-skycamp3)   -               2       2         -      0.00                
 Kubernetes (amd-devclou...8s-skycamp4)   -               2       2         -      0.00                
 Kubernetes (gke_skycamp...-api-server)   -               2       2         -      0.00                
 GCP (us-central1-a)                      n4-standard-8   8       32        -      0.36                
 AWS (us-east-1)                          m6i.2xlarge     8       32        -      0.38                
-------------------------------------------------------------------------------------------------------
Launching a new cluster 'hello-sky-user_2'. Proceed? [Y/n]: 
```

#### **2. Provision the cluster**

SkyPilot will setup a cluster with the requested resources and setup a SSH profile for it.


#### **3. Run the task's `setup` commands to prepare the cluster for running the task**

SkyPilot will run any commands specified in the `setup` field in the YAML on the VMs in the cluster. In this case, it will install the `cowsay` package.


#### **4. Run the task's `run` commands**

Finally, SkyPilot will run the commands specified in the `run` field. These commands can use any dependencies installed in the `setup` phase.

```console
(example, pid=1487) Hello SkyPilot!
(example, pid=1487)   ______________
(example, pid=1487) | Moo! SkyPilot! |
(example, pid=1487)   ==============
(example, pid=1487)               \
(example, pid=1487)                \
(example, pid=1487)                  ^__^
(example, pid=1487)                  (oo)\_______
(example, pid=1487)                  (__)\       )\/\
(example, pid=1487)                      ||----w |
(example, pid=1487)                      ||     ||
✓ Job finished (status: SUCCEEDED).
```

# Tasks and Clusters in SkyPilot

**Tasks** in SkyPilot are executed on **clusters**. A **cluster** is a collection of nodes on a cloud.

When you run a task with `sky launch`, SkyPilot creates a new cluster with a random name if an existing cluster is not specified.

> **💡 Hint** - When running `sky launch`, you can give the cluster a name with the `-c` flag. E.g. `sky launch -c mycluster example.yaml` would launch a cluster with the name `mycluster`. If the cluster name already exists, then SkyPilot will try to reuse the cluster by re-running the `setup` commands on the cluster.

You can see a table of your clusters with the command `sky status`.

## <span style="color:green">[DIY]</span> 💻 Checking your cluster status with `sky status`

**In a terminal window, run:**


-------------------------
```console
sky status
```
-------------------------

### Expected output
-------------------------
```console
root@c4784b202994:/skycamp-tutorial/01_hello_sky# sky status
Enabled Infra: kubernetes/amd-devcloud-mi300-k8s-skycamp1, kubernetes/amd-devcloud-mi300-k8s-skycamp2, kubernetes/amd-devcloud-mi300-k8s-skycamp3, kubernetes/amd-devcloud-mi300-k8s-skycamp4, kubernetes/gke_skycamp-skypilot-fastchat_us-central1-c_skycamp25-api-server, aws, gcp

Clusters
NAME              INFRA                                   RESOURCES               STATUS  AUTOSTOP  LAUNCHED     
hello-sky-user_2  Kubernetes (amd-devclou...8s-skycamp1)  1x(cpus=2, mem=2, ...)  UP      -         32 secs ago  

Managed jobs
No in-progress managed jobs.

Services
No live services. (See: sky serve -h)

* To see all managed jobs: sky jobs queue
```
-------------------------

We can see that the `sky launch` in the previous cells created a cluster with the name `hello-sky-$SKYPILOT_USER`.

## <span style="color:green">[DIY]</span> 💻 Connect to the cluster using SSH!

You can also easily SSH into a SkyPilot cluster with the `ssh` utility. 

**In a terminal window, run:**

-------------------------
```console
ssh hello-sky-$SKYPILOT_USER
```
-------------------------

### Expected output

This will drop you into an interactive terminal inside your cluster:

-------------------------
```console
root@c4784b202994:/skycamp-tutorial/01_hello_sky# ssh hello-sky-$SKYPILOT_USER
Warning: Permanently added '127.0.0.1' (ED25519) to the list of known hosts.
Warning: Permanently added '10.42.0.48' (ED25519) to the list of known hosts.
Linux hello-sky-user-2-81333a82-head 6.8.0-86-generic #87-Ubuntu SMP PREEMPT_DYNAMIC Mon Sep 22 18:03:36 UTC 2025 x86_64

The programs included with the Debian GNU/Linux system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Debian GNU/Linux comes with ABSOLUTELY NO WARRANTY, to the extent
permitted by applicable law.
(base) sky@hello-sky-user-2-81333a82-head:~$ hostname
hello-sky-user-2-81333a82-head
```
-------------------------

You can use `ctrl+d` to exit from the SSH session.

> **💡 Hint** - To enable the SSH functionality, SkyPilot adds the remote cluster to your `~/.ssh/config`. This means you can use the cluster name alias with other ssh tools, such as `scp`, `rsync`, VSCode and more!

# Cluster lifecycle management

SkyPilot clusters can exist in four states, each of which has different billing and storage implications:

* **`INIT`** - Cluster is initializing.
* **`UP`** - Cluster is up and running, you will be billed for the instance and the attached storages.
* **`STOPPED`** - Cluster nodes are shut down and their disks are suspended. Your data and node state is safe and the cluster can be restored to running state when required. You will be billed only for the storage.
* **`TERMINATED`** - Cluster is terminated and all nodes and their attached disks are deleted. These clusters cannot be restarted and will not be shown in `sky status`.

To manage these states, SkyPilot offers several useful commands:

1. **`sky stop`** - stops a `UP` cluster.
2. **`sky start`** - starts a `STOPPED` cluster.
2. **`sky down`** - terminates a `UP` or `STOPPED` cluster.
2. **`sky autostop`** - sets a cluster to automatically stop after a period of inactivity.

> **💡 Hint** - `sky stop` and `sky start` are useful when you want to suspend your experiments for a while but want to quickly resume later. `sky down` is useful to delete a cluster and restart a job from scratch.

## <span style="color:green">[DIY]</span> 💻 Terminate your cluster!
Now that we are done using the cluster, let's terminate it to stop being billed for it. You can use `sky down` to terminate a cluster.

**Run `sky down` to terminate the cluster**

-------------------------
```console
sky down hello-sky-$SKYPILOT_USER
```
-------------------------



### Expected output

-------------------------
```console
root@c4784b202994:/skycamp-tutorial/01_hello_sky# sky down hello-sky-$SKYPILOT_USER
Terminating 1 cluster: hello-sky-user_2. Proceed? [Y/n]: 
Terminating cluster hello-sky-user_2...done.
Terminating 1 cluster ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
```
-------------------------

#### 🎉 Congratulations! You have used SkyPilot to seamlessly run tasks on the cloud! Please proceed to the next notebook to learn how to use accelerators and object stores in SkyPilot.
